In [1]:
import numpy as np
import pandas as pd
from pandas_profiling import ProfileReport

In [2]:
df = pd.read_csv('Hot Stuff.csv')

In [3]:
df.head()

,url,WeekID,Week Position,Song,Performer,SongID,Instance,Previous Week Position,Peak Position,Weeks on Chart
0,http://www.billboard.com/charts/hot-100/1965-0...,7/17/1965,34,Don't Just Stand There,Patty Duke,Don't Just Stand TherePatty Duke,1,45.0,34,4
1,http://www.billboard.com/charts/hot-100/1965-0...,7/24/1965,22,Don't Just Stand There,Patty Duke,Don't Just Stand TherePatty Duke,1,34.0,22,5
2,http://www.billboard.com/charts/hot-100/1965-0...,7/31/1965,14,Don't Just Stand There,Patty Duke,Don't Just Stand TherePatty Duke,1,22.0,14,6
3,http://www.billboard.com/charts/hot-100/1965-0...,8/7/1965,10,Don't Just Stand There,Patty Duke,Don't Just Stand TherePatty Duke,1,14.0,10,7
4,http://www.billboard.com/charts/hot-100/1965-0...,8/14/1965,8,Don't Just Stand There,Patty Duke,Don't Just Stand TherePatty Duke,1,10.0,8,8


In [4]:
df['WeekID'] = df['WeekID'].astype('datetime64[ns]')

In [5]:
df1 = df[df['WeekID'] >= '2000-01-01']

In [6]:
spot = pd.read_excel('Hot 100 Audio Features.xlsx')

In [7]:
spot.head()

,SongID,Performer,Song,spotify_genre,spotify_track_id,spotify_track_preview_url,spotify_track_duration_ms,spotify_track_explicit,spotify_track_album,danceability,...,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,time_signature,spotify_track_popularity
0,-twistin'-White Silver SandsBill Black's Combo,Bill Black's Combo,-twistin'-White Silver Sands,[],NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,¿Dònde Està Santa Claus? (Where Is Santa Claus...,Augie Rios,¿Dònde Està Santa Claus? (Where Is Santa Claus?),['novelty'],NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,......And Roses And RosesAndy Williams,Andy Williams,......And Roses And Roses,"['adult standards', 'brill building pop', 'eas...",3tvqPPpXyIgKrm4PR9HCf0,https://p.scdn.co/mp3-preview/cef4883cfd1e0e53...,166106.0,0.0,The Essential Andy Williams,0.154,...,-14.063,1.0,0.0315,0.91100,0.000267,0.112,0.150,83.969,4.0,38.0
3,...And Then There Were DrumsSandy Nelson,Sandy Nelson,...And Then There Were Drums,"['rock-and-roll', 'space age pop', 'surf music']",1fHHq3qHU8wpRKHzhojZ4a,NaN,172066.0,0.0,Compelling Percussion,0.588,...,-17.278,0.0,0.0361,0.00256,0.745000,0.145,0.801,121.962,4.0,11.0
4,...Baby One More TimeBritney Spears,Britney Spears,...Baby One More Time,"['dance pop', 'pop', 'post-teen pop']",3MjUtNVVq3C8Fn0MP3zhXa,https://p.scdn.co/mp3-preview/da2134a161f1cb34...,211066.0,0.0,...Baby One More Time (Digital Deluxe Version),0.759,...,-5.745,0.0,0.0307,0.20200,0.000131,0.443,0.907,92.960,4.0,77.0


In [8]:
merged = pd.merge(df1, spot, on='SongID', how='inner')

In [9]:
merged = merged[merged['danceability'].notna()]

In [25]:
df2 = merged.sort_values(by=['WeekID'], ascending=True)
df2 = df2.drop_duplicates(subset='SongID', keep="first")

In [11]:
from urllib.request import urlopen
from bs4 import BeautifulSoup
from time import sleep
import csv
import json
import re

In [12]:
songs = df2['Song_x'].tolist()
artists = df2['Performer_x'].tolist()

a_zip = zip(artists, songs)
songList = list(a_zip)

In [13]:
songList

[('Eve', 'Gotta Man'),
 ('Jessica Simpson', 'I Wanna Love You Forever'),
 ('Chico DeBarge', 'Give You What You Want (Fa Sure)'),
 ('Rage Against The Machine', 'Guerrilla Radio'),
 ('Mint Condition', 'If You Love Me'),
 ("Ol' Dirty Bastard Featuring Kelis", 'Got Your Money'),
 ('Nas', 'NAStradamus'),
 ('M2M', "Don't Say You Love Me"),
 ('LFO', 'Girl On TV'),
 ('Beth Hart', 'L.A. Song'),
 ('Sisqo Featuring Make It Hot', 'Got To Get It'),
 ('Will Smith Featuring K-Ci', 'Will 2K'),
 ('IMx', 'Stay The Night'),
 ('Savage Garden', 'I Knew I Loved You'),
 ('Ty Herndon', 'Steam'),
 ('Ice Cube Featuring Mack 10 & Ms. Toi', 'You Can Do It'),
 ('Limp Bizkit Featuring Method Man', 'N 2 Gether Now'),
 ('Clay Walker', 'Live, Laugh, Love'),
 ('Tim McGraw', 'My Best Friend'),
 ('Blink-182', 'All The Small Things'),
 ("Lil' Wayne Featuring Juvenile & B.G.", 'Tha Block Is Hot'),
 ('Mariah Carey Featuring Joe & 98 Degrees', 'Thank God I Found You'),
 ('Celine Dion', "That's The Way It Is"),
 ('702', 'Wher

In [14]:
import random
from urllib.request import urlopen
from bs4 import BeautifulSoup
from time import sleep
import csv
import json
import re
import requests
import urllib.request
from fuzzywuzzy import fuzz
from fuzzywuzzy import process



def scrapeLyrics2(songList):
    lyricsList = []
    searched = []
    count = 0
    for pair in songList:
        print(count)
        artist = pair[0].lower()
        if artist[0:3] == "the":
            artist = artist.replace("the", "", 1)
        sep = 'featuring'
        artist = artist.split(sep, 1)[0]
        sep = '&'
        artist = artist.split(sep, 1)[0]
        artist = artist.replace(" ", "")
        artist = ''.join(e for e in artist if e.isalnum())
    
        song = pair[1].lower().replace(" ", "")
        song = ''.join(e for e in song if e.isalnum())
        # url to scrape the lyrics from
        base_url = "https://www.azlyrics.com/lyrics/{}/{}.html"


        lyrics_not_found_for = []

        # delay after each execution of call for not exceeding the requests count and also not to overburden the server
        delay = 10


        #artist = 'beatles'
        #song = "cometogether"
        final_url = base_url.format(artist,song)


        user_agent_list = [
        'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_5) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/13.1.1 Safari/605.1.15',
        'Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:77.0) Gecko/20100101 Firefox/77.0',
        'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_5) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/83.0.4103.97 Safari/537.36',
        'Mozilla/5.0 (Macintosh; Intel Mac OS X 10.15; rv:77.0) Gecko/20100101 Firefox/77.0',
        'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/83.0.4103.97 Safari/537.36',
        ]

        for i in range(1,4):
        #Pick a random user agent
            user_agent = random.choice(user_agent_list)
        #Set the headers 
        headers = {'User-Agent': user_agent}


        try:
            html_page = requests.get(final_url, headers=headers)

            page_html = html_page.text
            soup = BeautifulSoup(page_html, 'html.parser')

            html_pointer = soup.find('div', attrs={'class':'ringtone'})
            song_name = html_pointer.find_next('b').contents[0].strip()
            lyrics = html_pointer.find_next('div').text.strip()


            print("Lyrics successfully written to file for : " + song_name)
            lyricsList.append(lyrics)
            searched.append(0)

        except:
            try:
                searchURL = 'https://search.azlyrics.com/search.php?q='
                newURL = searchURL + pair[1].replace(" ", "+")
                    
                page_html = urllib.request.urlopen(newURL)

                soup = BeautifulSoup(page_html, 'html.parser', from_encoding=page_html.info().get_param('charset'))

                linkList = []
                for link in soup.find_all('a', href=True):
                    linkList.append(link['href'])
                    
                new_final_url = process.extractOne(final_url, linkList)[0]
                
                html_page = requests.get(new_final_url, headers=headers)
                page_html = html_page.text
                soup = BeautifulSoup(page_html, 'html.parser')

                html_pointer = soup.find('div', attrs={'class':'ringtone'})
                song_name = html_pointer.find_next('b').contents[0].strip()
                lyrics = html_pointer.find_next('div').text.strip()


                print("Lyrics successfully written to file for : " + song_name)
                print("searched")
                lyricsList.append(lyrics)
                searched.append(1)
                
                
            except:
                print("Lyrics not found for : ", final_url)
                lyricsList.append('No lyrics')

        finally:
            sleep(delay)
        
        count+=1
    return [lyricsList, searched]

/Users/mhgrody/opt/anaconda3/lib/python3.8/site-packages/fuzzywuzzy/fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')


In [15]:
lyr = scrapeLyrics2(songList[2000:4000])

0
Lyrics successfully written to file for : "Every Mile A Memory"
1
Lyrics successfully written to file for : "Money Maker"
2
Lyrics successfully written to file for : "When You Were Young"
3
Lyrics successfully written to file for : "That Girl"
4
Lyrics successfully written to file for : "I Loved Her First"
5
Lyrics successfully written to file for : "Say Goodbye"
6
Lyrics successfully written to file for : "Entourage"
7
Lyrics successfully written to file for : "Show Me The Money"
8
Lyrics successfully written to file for : "Tell Me Baby"
9
Lyrics not found for :  https://www.azlyrics.com/lyrics/game/itsokayoneblood.html
10
Lyrics successfully written to file for : "Show Stopper"
11
Lyrics successfully written to file for : "Too Little, Too Late"
12
Lyrics successfully written to file for : "Call Me When You're Sober"
13
Lyrics successfully written to file for : "Feels Just Like It Should"
14
Lyrics successfully written to file for : "Mountains"
15
Lyrics successfully written to file

In [16]:
songList

[('Eve', 'Gotta Man'),
 ('Jessica Simpson', 'I Wanna Love You Forever'),
 ('Chico DeBarge', 'Give You What You Want (Fa Sure)'),
 ('Rage Against The Machine', 'Guerrilla Radio'),
 ('Mint Condition', 'If You Love Me'),
 ("Ol' Dirty Bastard Featuring Kelis", 'Got Your Money'),
 ('Nas', 'NAStradamus'),
 ('M2M', "Don't Say You Love Me"),
 ('LFO', 'Girl On TV'),
 ('Beth Hart', 'L.A. Song'),
 ('Sisqo Featuring Make It Hot', 'Got To Get It'),
 ('Will Smith Featuring K-Ci', 'Will 2K'),
 ('IMx', 'Stay The Night'),
 ('Savage Garden', 'I Knew I Loved You'),
 ('Ty Herndon', 'Steam'),
 ('Ice Cube Featuring Mack 10 & Ms. Toi', 'You Can Do It'),
 ('Limp Bizkit Featuring Method Man', 'N 2 Gether Now'),
 ('Clay Walker', 'Live, Laugh, Love'),
 ('Tim McGraw', 'My Best Friend'),
 ('Blink-182', 'All The Small Things'),
 ("Lil' Wayne Featuring Juvenile & B.G.", 'Tha Block Is Hot'),
 ('Mariah Carey Featuring Joe & 98 Degrees', 'Thank God I Found You'),
 ('Celine Dion', "That's The Way It Is"),
 ('702', 'Wher

In [18]:
len(lyr[0])

2000

In [20]:
count = 0
counts = []

for song in lyr[0]:
    if song == 'No lyrics':
        counts.append(count)
    count+=1
        
len(counts)

79

In [39]:
counts[-4]

362

In [40]:
songList[362]

('Mos Def & Pharoahe Monch Featuring Nate Dogg', 'Oh No')

In [44]:
testList = scrapeLyrics2(songList[360:365])

Lyrics successfully written to file for : "Liquid Dreams"
Lyrics successfully written to file for : "There Is No Arizona"
Lyrics successfully written to file for : "Oh No"
Lyrics successfully written to file for : "Angel"
Lyrics successfully written to file for : "X"


In [29]:
df3 = df2[2000:4000]
df3['Lyrics'] = lyr[0]
df3

<ipython-input-29-fcfb405a16e9>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df3['Lyrics'] = lyr[0]


,url,WeekID,Week Position,Song_x,Performer_x,SongID,Instance,Previous Week Position,Peak Position,Weeks on Chart,...,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,time_signature,spotify_track_popularity,Lyrics
92057,http://www.billboard.com/charts/hot-100/2006-0...,2006-08-26,95,Every Mile A Memory,Dierks Bentley,Every Mile A MemoryDierks Bentley,1,NaN,95,1,...,1.0,0.0356,0.080300,0.001310,0.162,0.392,177.915,4.0,44.0,"Country Roads, old theater marquee signs\nPark..."
93658,http://www.billboard.com/charts/hot-100/2006-0...,2006-08-26,96,Money Maker,Ludacris Featuring Pharrell,Money MakerLudacris Featuring Pharrell,1,NaN,96,1,...,1.0,0.2520,0.177000,0.000000,0.216,0.583,83.317,4.0,62.0,[Hook (Pharrell / Ludacris):]\nShake your mone...
107582,http://www.billboard.com/charts/hot-100/2006-0...,2006-08-26,29,When You Were Young,The Killers,When You Were YoungThe Killers,1,NaN,29,1,...,1.0,0.1120,0.000152,0.048400,0.280,0.321,130.435,4.0,76.0,You sit there in your heartache\nWaiting on so...
66993,http://www.billboard.com/charts/hot-100/2006-0...,2006-08-26,100,That Girl,Frankie J Featuring Mannie Fresh & Chamillionaire,That GirlFrankie J Featuring Mannie Fresh & Ch...,1,NaN,100,1,...,0.0,0.1720,0.206000,0.000000,0.174,0.569,170.138,4.0,38.0,[Intro - Mannie Fresh (Chamillionaire):]\r\nLa...
58908,http://www.billboard.com/charts/hot-100/2006-0...,2006-08-26,75,I Loved Her First,Heartland,I Loved Her FirstHeartland,1,NaN,75,1,...,1.0,0.0262,0.295000,0.000000,0.175,0.166,137.031,3.0,60.0,Look at the two of you dancing that way\nLost ...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
76403,http://www.billboard.com/charts/hot-100/2011-1...,2011-10-15,83,Fight For You,Jason Derulo,Fight For YouJason Derulo,1,NaN,83,1,...,1.0,0.0787,0.013900,0.000000,0.113,0.523,177.942,4.0,58.0,It's gonna take a lot to drag me away from you...
42420,http://www.billboard.com/charts/hot-100/2011-1...,2011-10-15,88,After Midnight,Blink-182,After MidnightBlink-182,1,NaN,88,1,...,1.0,0.0490,0.006420,0.000017,0.020,0.902,167.995,4.0,54.0,[Tom DeLonge:]\nI can't get my feet up off the...
4247,http://www.billboard.com/charts/hot-100/2011-1...,2011-10-15,7,Strange Clouds,B.o.B Featuring Lil Wayne,Strange CloudsB.o.B Featuring Lil Wayne,1,NaN,7,1,...,0.0,0.2860,0.060700,0.000000,0.131,0.800,73.726,4.0,61.0,[Intro: B.o.B]\nYeah\nHahaha\nOh you already k...
36414,http://www.billboard.com/charts/hot-100/2011-1...,2011-10-15,81,Dedication To My Ex (Miss That),Lloyd Featuring Andre 3000 & Lil Wayne,Dedication To My Ex (Miss That)Lloyd Featuring...,1,NaN,81,1,...,1.0,0.0592,0.018200,0.000007,0.107,0.964,119.016,4.0,50.0,[Lil Wayne]\nHey ya\nI came to talk about this...


In [30]:
df3.to_csv('new data with AZLyrics.csv')